In [150]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import re
pd.set_option("display.max_columns", None)

In [151]:
def unzip(path_to_zip_file, directory_to_extract_to):
    """
    Args:
        path_to_zip_file (str): 압축파일 경로
        directory_to_extract_to (str): 압축 풀고 저장할 경로
    """
    import zipfile
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)


In [152]:
from google.colab import drive
drive.mount("/content/drive")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [153]:
import shutil
shutil.copyfile("/content/drive/MyDrive/고려대학교/기계학습개론/url_encoded_dataset.csv", "/content/url_encoded_dataset.csv")
shutil.copyfile("/content/drive/MyDrive/고려대학교/기계학습개론/tcp_payload_RNN.model.zip", "/content/tcp_payload_RNN.model.zip")
shutil.copyfile("/content/drive/MyDrive/고려대학교/기계학습개론/http_data_RNN.model.zip", "/content/http_data_RNN.model.zip")

'/content/http_data_RNN.model.zip'

In [154]:
data = pd.read_csv("/content/url_encoded_dataset.csv").sample(frac=1)
data = data.drop(["Unnamed: 0", "http.request.uri.query", "http.file_data", "http.request.full_uri", 
                  "tcp.options", "http.request.uri.path", "http.request.uri.query.dict", "http.file_data.dict", "http.request.uri.scheme"], axis="columns")
data["http.data"] = data["http.data"].fillna("")
original_data = data.copy()

# RNN학습시, URL정보를 바탕으로 분류를 진행할 수 있으므로 URL 정보를 삭제해야함.
data["tcp.payload"] = original_data["tcp.payload"].astype("str").apply(lambda x: " ".join([_ for _ in x.split(" ") if not (_.startswith("/DVWA") and _.endswith(".php"))]))

<ipython-input-154-776e6cc6f1c7>:1: DtypeWarning: Columns (3,6,10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/url_encoded_dataset.csv").sample(frac=1)


## RNN을 위한 전처리

In [255]:
# Tokenize values
tokenizer = Tokenizer(oov_token="<NUMBER>", filters='!"#$%&()*+,-./:?@[\\]^_`{|}~\t\n=',)
tokenizer.fit_on_texts(original_data["http.data"])
padded = pad_sequences(tokenizer.texts_to_sequences(original_data["http.data"]), padding="post", value=0)
data["http.data"] = padded.tolist()

In [256]:
word_index_dict = {val: key for key, val in tokenizer.word_index.items()}
pd.Series(tokenizer.texts_to_sequences(original_data["http.data"])).apply(lambda x: ", ".join([(word_index_dict)[_] for _ in x])).unique()

array(['', 'username, admin, password, 00, login, login',
       '0, 0, id, 4, 20and, 201970, 20in, 20, 28select, 20, 28char, 28113, 29, 2bchar, 2898, 29, 2bchar, 28107, 29, 2bchar, 28107, 29, 2bchar, 28113, 29, 2b, 28select, 20, 28case, 20when, 20, 281970, 3d1970, 29, 20then, 20char, 2849, 29, 20else, 20char, 2848, 29, 20end, 29, 29, 2bchar, 28113, 29, 2bchar, 28107, 29, 2bchar, 28107, 29, 2bchar, 28107, 29, 2bchar, 28113, 29, 29, 29, 20zvuw, submit, submitid, 55',
       ...,
       '0, 0, id, 7, submit, submitid, 3d57, 20and, 201412, 20in, 20, 28select, 20, 28char, 28113, 29, 2bchar, 28107, 29, 2bchar, 28122, 29, 2bchar, 28107, 29, 2bchar, 28113, 29, 2b, 28select, 20, 28case, 20when, 20, 281412, 3d1412, 29, 20then, 20char, 2849, 29, 20else, 20char, 2848, 29, 20end, 29, 29, 2bchar, 28113, 29, 2bchar, 28112, 29, 2bchar, 28106, 29, 2bchar, 28120, 29, 2bchar, 28113, 29, 29, 29, 20djzl',
       '0, 0, id, 2, 29, 20and, 203708, 3ddbms, pipe, receive, message, 28chr, 28105, 29, 7c, 7cchr, 

In [156]:
# Tokenize tcp.payload
tokenizer_payload = Tokenizer(oov_token="<PAD>")
tokenizer_payload.fit_on_texts(original_data["tcp.payload"].astype("str"))
padded = pad_sequences(tokenizer_payload.texts_to_sequences(original_data["tcp.payload"].astype("str")), padding="post", value=0)
data["tcp.payload"] = padded.tolist()

In [157]:
data.head()

,ip.src_host,frame.time,tcp.checksum,http.response,arp.src.proto_ipv4,http.content_length,Attack_type,arp.dst.proto_ipv4,tcp.connection.rst,arp.hw.size,tcp.len,tcp.seq,tcp.connection.syn,http.referer,ip.dst_host,tcp.flags.ack,tcp.ack_raw,tcp.connection.fin,tcp.dstport,tcp.flags,tcp.ack,tcp.connection.synack,tcp.payload,tcp.srcport,arp.opcode,0,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,http.request.uri.netloc,http.data
18879,192.168.0.170,2021 17:42:20.919856000,0x00004a16,0.0,0,0.0,Password,0,0.0,0.0,86.0,1.0,0.0,0.0,192.168.0.128,1.0,6.752434e+08,0.0,80.0,0x00000018,1.0,0.0,"[18, 10, 3, 15, 6, 5, 2, 11, 7, 8, 2, 9, 12, 1...",50896.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
35241,192.168.0.170,2021 18:47:06.547787000,0x00009bff,0.0,0,0.0,Password,0,0.0,0.0,86.0,1.0,0.0,0.0,192.168.0.128,1.0,1.438164e+07,0.0,80.0,0x00000018,1.0,0.0,"[18, 10, 3, 15, 6, 5, 2, 11, 7, 8, 2, 9, 12, 1...",40796.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
28744,192.168.0.170,2021 18:25:19.844375000,0x0000b8e5,0.0,0,38.0,Password,0,0.0,0.0,261.0,1.0,0.0,0.0,192.168.0.128,1.0,2.248116e+08,0.0,80.0,0x00000018,1.0,0.0,"[31, 10, 3, 15, 6, 5, 2, 11, 7, 8, 2, 9, 12, 1...",50604.0,0.0,False,False,True,False,True,False,192.168.0.128,"[5, 6, 7, 8, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
64730,192.168.0.170,2021 20:26:01.618398000,0x00004f26,0.0,0,0.0,Password,0,0.0,0.0,221.0,1.0,0.0,0.0,192.168.0.128,1.0,3.593927e+09,0.0,80.0,0x00000018,1.0,0.0,"[18, 10, 3, 15, 6, 5, 2, 11, 7, 8, 2, 9, 12, 1...",39724.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
55548,192.168.0.170,2021 19:55:37.495345000,0x000006b2,0.0,0,0.0,Password,0,0.0,0.0,221.0,1.0,0.0,0.0,192.168.0.128,1.0,1.187137e+09,0.0,80.0,0x00000018,1.0,0.0,"[18, 10, 3, 15, 6, 5, 2, 11, 7, 8, 2, 9, 12, 1...",41942.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [158]:
original_data.head()

,ip.src_host,frame.time,tcp.checksum,http.response,arp.src.proto_ipv4,http.content_length,Attack_type,arp.dst.proto_ipv4,tcp.connection.rst,arp.hw.size,tcp.len,tcp.seq,tcp.connection.syn,http.referer,ip.dst_host,tcp.flags.ack,tcp.ack_raw,tcp.connection.fin,tcp.dstport,tcp.flags,tcp.ack,tcp.connection.synack,tcp.payload,tcp.srcport,arp.opcode,0,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,http.request.uri.netloc,http.data
18879,192.168.0.170,2021 17:42:20.919856000,0x00004a16,0.0,0,0.0,Password,0,0.0,0.0,86.0,1.0,0.0,0.0,192.168.0.128,1.0,6.752434e+08,0.0,80.0,0x00000018,1.0,0.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,50896.0,0.0,False,True,False,False,True,False,192.168.0.128,
35241,192.168.0.170,2021 18:47:06.547787000,0x00009bff,0.0,0,0.0,Password,0,0.0,0.0,86.0,1.0,0.0,0.0,192.168.0.128,1.0,1.438164e+07,0.0,80.0,0x00000018,1.0,0.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,40796.0,0.0,False,True,False,False,True,False,192.168.0.128,
28744,192.168.0.170,2021 18:25:19.844375000,0x0000b8e5,0.0,0,38.0,Password,0,0.0,0.0,261.0,1.0,0.0,0.0,192.168.0.128,1.0,2.248116e+08,0.0,80.0,0x00000018,1.0,0.0,POST /DVWA/login.php HTTP/1.0\nHost: 192.168.0...,50604.0,0.0,False,False,True,False,True,False,192.168.0.128,username=admin&password=00&Login=Login
64730,192.168.0.170,2021 20:26:01.618398000,0x00004f26,0.0,0,0.0,Password,0,0.0,0.0,221.0,1.0,0.0,0.0,192.168.0.128,1.0,3.593927e+09,0.0,80.0,0x00000018,1.0,0.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,39724.0,0.0,False,True,False,False,True,False,192.168.0.128,
55548,192.168.0.170,2021 19:55:37.495345000,0x000006b2,0.0,0,0.0,Password,0,0.0,0.0,221.0,1.0,0.0,0.0,192.168.0.128,1.0,1.187137e+09,0.0,80.0,0x00000018,1.0,0.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,41942.0,0.0,False,True,False,False,True,False,192.168.0.128,


In [159]:
def zipFile(fileName):
    import shutil
    shutil.make_archive(fileName,"zip", fileName)

## DNN을 위한 전처리

In [160]:
# process time data into hour
def time_to_sec(time):
    hour, minute, second = list(map(float, time[5:].strip().split(":")))
    sec = hour*3600 + minute*60 + second
    return sec / 3600
data["frame.time"] = original_data["frame.time"].apply(time_to_sec)

In [161]:
# tcp.checksum float로 변경
data["tcp.checksum"] = original_data["tcp.checksum"].astype("str").apply(lambda x: int(x[2:] if x[2:] else "0", 16))

In [162]:
# tcp.flag float로 변경
data["tcp.flags"] = original_data["tcp.flags"].astype("str").apply(lambda x: int(x[2:] if x[2:] else "0", 16))

In [163]:
# tcp.ack_raw 정규화
data["tcp.ack_raw"] = (original_data["tcp.ack_raw"] - original_data["tcp.ack_raw"].mean()) / original_data["tcp.ack_raw"].std()

In [164]:
# tcp.ack 정규화
data["tcp.ack"] = (original_data["tcp.ack"] - original_data["tcp.ack"].mean()) / original_data["tcp.ack"].std()

In [176]:
for feature in ["GET", "POST", "TRACE", "HTTP/1.0", "HTTP/1.1"]:
    data[feature] = original_data[feature].apply(int)

In [202]:
# http.referer는 [0.0, '0.0', '0', '127.0.0.1'] 의 값만을 갖고 있으므로, 의미가 없음.
data = data.drop(["http.referer"], axis="columns") 

In [ ]:
data = data.drop(["0"], axis="columns")

In [204]:
# DNN을 할 columns
DNN_COLS = ['frame.time', 'tcp.checksum', 'http.response',
       'http.content_length',
       'tcp.connection.rst', 'arp.hw.size', 'tcp.len',
       'tcp.seq', 'tcp.connection.syn',
       'tcp.flags.ack', 'tcp.ack_raw', 'tcp.connection.fin', 'tcp.dstport',
       'tcp.flags', 'tcp.ack', 'tcp.connection.synack', 
       'tcp.srcport', 'arp.opcode', 'GET', 'POST', 'TRACE', 'HTTP/1.0',
       'HTTP/1.1']
data[DNN_COLS].head(10)

,frame.time,tcp.checksum,http.response,http.content_length,tcp.connection.rst,arp.hw.size,tcp.len,tcp.seq,tcp.connection.syn,tcp.flags.ack,tcp.ack_raw,tcp.connection.fin,tcp.dstport,tcp.flags,tcp.ack,tcp.connection.synack,tcp.srcport,arp.opcode,GET,POST,TRACE,HTTP/1.0,HTTP/1.1
18879,17.705811,18966,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,-1.111343,0.0,80.0,24,-0.26364,0.0,50896.0,0.0,1,0,0,1,0
35241,18.785152,39935,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,-1.630681,0.0,80.0,24,-0.26364,0.0,40796.0,0.0,1,0,0,1,0
28744,18.422179,47333,0.0,38.0,0.0,0.0,261.0,1.0,0.0,1.0,-1.465315,0.0,80.0,24,-0.26364,0.0,50604.0,0.0,0,1,0,1,0
64730,20.433783,20262,0.0,0.0,0.0,0.0,221.0,1.0,0.0,1.0,1.182305,0.0,80.0,24,-0.26364,0.0,39724.0,0.0,1,0,0,1,0
55548,19.927082,1714,0.0,0.0,0.0,0.0,221.0,1.0,0.0,1.0,-0.709072,0.0,80.0,24,-0.26364,0.0,41942.0,0.0,1,0,0,1,0
48384,19.527418,55511,0.0,38.0,0.0,0.0,261.0,1.0,0.0,1.0,0.248223,0.0,80.0,24,-0.26364,0.0,49782.0,0.0,0,1,0,1,0
24282,18.087522,63512,0.0,0.0,0.0,0.0,221.0,1.0,0.0,1.0,-0.196100,0.0,80.0,24,-0.26364,0.0,37968.0,0.0,1,0,0,1,0
47766,19.492175,8552,0.0,38.0,0.0,0.0,261.0,1.0,0.0,1.0,0.146197,0.0,80.0,24,-0.26364,0.0,48048.0,0.0,0,1,0,1,0
65858,20.495657,43138,0.0,0.0,0.0,0.0,221.0,1.0,0.0,1.0,1.494050,0.0,80.0,24,-0.26364,0.0,42928.0,0.0,1,0,0,1,0
15676,17.525967,52757,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,-0.150152,0.0,80.0,24,-0.26364,0.0,41784.0,0.0,1,0,0,1,0


## 데이터 분할

In [257]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.3)

In [258]:
labels = {key: i for i, key in enumerate(test["Attack_type"].unique())}
def to_onehot(x):
    l = [0 for _ in range(labels.__len__())]
    l[labels[x]] = 1
    return l
test_X, test_Y = test.drop(["Attack_type"], axis="columns"), np.stack(test["Attack_type"].apply(to_onehot))
train_X, train_Y =  train.drop(["Attack_type"], axis="columns"), np.stack(train["Attack_type"].apply(to_onehot))

## RNN 실행

In [259]:
payload_test_X ,RNN_test_X =  np.stack(test["tcp.payload"]),np.stack(test["http.data"])
payload_train_X ,RNN_train_X =  np.stack(train["tcp.payload"]),np.stack(train["http.data"])

In [260]:
vocab_size = tokenizer.word_index.keys().__len__()
input_dim = RNN_train_X.shape[1]

In [281]:
# layers=[
# keras.layers.Embedding(1000, 128, input_length=1000),
# keras.layers.LSTM(64, activation="tanh", dropout=0.5, return_sequences=True),
# # keras.layers.Dense(100, activation="relu"),
# # keras.layers.Dense(100, activation="relu"),
# keras.layers.Dense(3, activation="softmax")
# ]
import os
RNN_MODEL_PATH:str = "http_data_RNN.model"
FORCE_TRAIN = True
if "http_data_RNN.model.zip" in os.listdir(os.getcwd()) and not FORCE_TRAIN:
    unzip(f"{RNN_MODEL_PATH}.zip", f"{RNN_MODEL_PATH}")
    RNN_model = keras.models.load_model(f"{RNN_MODEL_PATH}")
else:
    RNN_model = keras.Sequential(
        layers = [
            keras.layers.Embedding(vocab_size, 128, input_length=input_dim),
            keras.layers.LSTM(128, dropout=0.7, activation="tanh"),
            keras.layers.Dense(100, activation="relu"),
            keras.layers.Dense(4, activation='softmax'),
        ]
    )
    RNN_model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(), 
        optimizer=tf.keras.optimizers.Adam(learning_rate=.005), 
        metrics=["accuracy"],
    )
    RNN_model.fit(RNN_train_X, train_Y, epochs=10, batch_size=128)
    RNN_model.save("http_data_RNN.model")
    zipFile("http_data_RNN.model")

Epoch 1/10
510/510 [==============================] - 23s 42ms/step - loss: 0.5655 - accuracy: 0.8042
Epoch 2/10
510/510 [==============================] - 7s 13ms/step - loss: 0.5472 - accuracy: 0.8088
Epoch 3/10
510/510 [==============================] - 5s 9ms/step - loss: 0.5525 - accuracy: 0.8088
Epoch 4/10
510/510 [==============================] - 3s 7ms/step - loss: 0.5106 - accuracy: 0.8084
Epoch 5/10
510/510 [==============================] - 3s 6ms/step - loss: 0.4751 - accuracy: 0.8163
Epoch 6/10
510/510 [==============================] - 4s 7ms/step - loss: 0.3659 - accuracy: 0.8420
Epoch 7/10
510/510 [==============================] - 4s 7ms/step - loss: 0.3654 - accuracy: 0.8420
Epoch 8/10
510/510 [==============================] - 3s 7ms/step - loss: 0.3652 - accuracy: 0.8420
Epoch 9/10
510/510 [==============================] - 3s 6ms/step - loss: 0.3652 - accuracy: 0.8420
Epoch 10/10
510/510 [==============================] - 3s 7ms/step - loss: 0.3651 - accuracy: 0.8

In [230]:
vocab_size = tokenizer_payload.word_index.keys().__len__()
input_dim = payload_train_X.shape[1]

In [ ]:
PAYLOAD_RNN_MODEL_PATH:str = "logs/tcp_payload_RNN.model"
if "tcp_payload_RNN.model.zip" in os.listdir(os.getcwd()):
    unzip(f"{PAYLOAD_RNN_MODEL_PATH}.zip", f"{PAYLOAD_RNN_MODEL_PATH}")
    payload_RNN_model = keras.models.load_model(f"{PAYLOAD_RNN_MODEL_PATH}")
else:
    payload_RNN_model = keras.Sequential(
        layers = [
            keras.layers.Embedding(vocab_size, 128, input_length=input_dim),
            keras.layers.LSTM(128, dropout=0.7, recurrent_dropout=0.7),
            keras.layers.Dense(100, activation="relu"),
            keras.layers.Dense(4, activation='softmax'),
        ]
    )
    payload_RNN_model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
        optimizer="adam", 
        metrics=["accuracy"],
    )
    payload_RNN_model.fit(payload_train_X, train_Y, epochs=5, batch_size=32)
    payload_RNN_model.save("tcp_payload_RNN.model")
    zipFile("tcp_payload_RNN.model")

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


 665/2040 [========>.....................] - ETA: 9:56 - loss: 0.7321 - accuracy: 0.7635

In [282]:
file_data_prediction = RNN_model.predict(RNN_test_X)

875/875 [==============================] - 3s 3ms/step


In [ ]:
payload_prediction = payload_RNN_model.predict(payload_test_X)

In [269]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

In [284]:
# http.file_data Prediction
precision_score(np.argmax(test_Y, axis=1), np.argmax(file_data_prediction, axis=1), average="macro")

0.9567758821248546

In [285]:
recall_score(np.argmax(test_Y, axis=1), np.argmax(file_data_prediction, axis=1), average="macro")

0.7677120233381954

In [283]:
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(file_data_prediction, axis=1)),index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))

,PRED_Password,PRED_XSS,PRED_SQL_injection,PRED_Uploading
TRUE_Password,21331,0,0,0
TRUE_XSS,4459,340,0,0
TRUE_SQL_injection,0,0,1298,0
TRUE_Uploading,0,0,0,543


In [ ]:
# tcp.payload Prediction
precision_score(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1), average="macro")

In [ ]:
recall_score(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1), average="macro")

In [ ]:
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1)),index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))

In [ ]:
shutil.copyfile("/content/tcp_payload_RNN.model.zip", "/content/drive/MyDrive/고려대학교/기계학습개론/tcp_payload_RNN.model.zip")
shutil.copyfile("/content/http_data_RNN.model.zip", "/content/drive/MyDrive/고려대학교/기계학습개론/http_data_RNN.model.zip")

## DNN 실행

In [293]:
DNN_model = keras.Sequential(
    layers=[
        keras.layers.Dense(500, activation="relu"),
        keras.layers.Dense(50, activation="relu"),
        keras.layers.Dense(50, activation="relu"),
        keras.layers.Dense(50, activation="relu"),
        keras.layers.Dense(50, activation="relu"),
        keras.layers.Dense(50, activation="relu"),
        keras.layers.Dense(50, activation="relu"),
        keras.layers.Dense(len(labels.keys()), activation="softmax"),
    ]
)
DNN_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
    optimizer="adam", 
    metrics=["accuracy"],
)
DNN_model.fit(train_X[DNN_COLS], train_Y, epochs=5, batch_size=30)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2176/2176 [==============================] - 11s 4ms/step - loss: 2.0504 - accuracy: 0.8232
Epoch 2/5
2176/2176 [==============================] - 9s 4ms/step - loss: 0.5635 - accuracy: 0.8459
Epoch 3/5
2176/2176 [==============================] - 8s 4ms/step - loss: 0.5638 - accuracy: 0.8459
Epoch 4/5
2176/2176 [==============================] - 8s 4ms/step - loss: 0.5639 - accuracy: 0.8459
Epoch 5/5
2176/2176 [==============================] - 8s 4ms/step - loss: 0.5638 - accuracy: 0.8459


In [294]:
DNN_prediction = DNN_model.predict(test_X[DNN_COLS])

875/875 [==============================] - 1s 1ms/step


In [224]:
precision_score(np.argmax(test_Y, axis=1), np.argmax(DNN_prediction, axis=1), average="macro")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.45905661115020724

In [225]:
recall_score(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1), average="macro")

0.250831313437358

In [226]:
confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1))

array([[ 1230,     0,    58,     0],
       [20372,     0,  1017,     0],
       [ 4487,     0,   228,     0],
       [  546,     0,    33,     0]])

In [295]:
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(DNN_prediction, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))

,PRED_Password,PRED_XSS,PRED_SQL_injection,PRED_Uploading
TRUE_Password,21331,0,0,0
TRUE_XSS,2469,2330,0,0
TRUE_SQL_injection,1298,0,0,0
TRUE_Uploading,543,0,0,0
